## How to use Itinerary Builder

Itinerary Builder is a module used to query a database of possible itineraries, and return a dataframe with information on each. The critical outputs include the origins and destinations of each two flight itinerary, their times, airlines, and durations. Included as well is the next best second leg flight for each if the connecting flight is missed.

A machine learning algorithm is used to weight the time cost of a missed connection based on the liklihood of the missed connnection occuring.

Users of the module must provide a connection time assumption. This is the minimum time between connecting flights which the user finds acceptable (e.g., allow 45 minutes minimum for connection).

Examples of each function's usage is below.

In [1]:
from itineraryBuilder import *

In [2]:
pd.set_option('display.max_columns', None)

### Itinerary Builder main function call

With default options, the resulting dataframe is ordered by risk as shown below.

In [3]:
origin = 'ATL'
destination = 'SEA'
flight_date = '8/3/2022'

dne = '700'
dnl = '1000'
ane = '1'
anl = '2000'
tc = 45

df = itineraryBuilder('faa', origin, destination, flight_date, tc, dne, dnl, ane, anl); df.head(10)

,FIRST_LEG_AIRLINE,FIRST_LEG_ORIG,FIRST_LEG_ORIG_CITY,FIRST_LEG_DEST,FIRST_LEG_DEST_CITY,FIRST_LEG_DATE,FIRST_LEG_FLIGHT_NUM,FIRST_LEG_DEP_TIME,FIRST_LEG_ARR_TIME,SECOND_LEG_AIRLINE,SECOND_LEG_ORIG,SECOND_LEG_ORIG_CITY,SECOND_LEG_DEST,SECOND_LEG_DEST_CITY,SECOND_LEG_DATE,SECOND_LEG_FLIGHT_NUM,SECOND_LEG_DEP_TIME,SECOND_LEG_ARR_TIME,NEXT_BEST_SECOND_LEG_DATE,NEXT_BEST_SECOND_LEG_DEP_TIME,NEXT_BEST_SECOND_LEG_ARR_TIME,FIRST_LEG_ORIG_TZ,FIRST_LEG_DEST_TZ,SECOND_LEG_ORIG_TZ,SECOND_LEG_DEST_TZ,FIRST_LEG_DEP_TIMESTAMP,FIRST_LEG_ARR_TIMESTAMP,SECOND_LEG_DEP_TIMESTAMP,SECOND_LEG_ARR_TIMESTAMP,NEXT_BEST_SECOND_LEG_DEP_TIMESTAMP,NEXT_BEST_SECOND_LEG_ARR_TIMESTAMP,overnight_bool_1,overnight_bool_2,overnight_bool_3,FIRST_FLIGHT_DURATION,SECOND_FLIGHT_DURATION,CONNECT_TIME,TRIP_TIME,RISK_MISSED_CONNECTION,NEXT_FLIGHT_TIMELOSS,TOTAL_RISK
0,DL,ATL,"Atlanta, GA",LAX,"Los Angeles, CA",8/3/2022,345,845,1020,DL,LAX,"Los Angeles, CA",SEA,"Seattle, WA",8/3/2022,803,1220,1505,8/3/2022,1402,1647,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2022-08-03 04:45:00-04:00,2022-08-03 03:20:00-07:00,2022-08-03 05:20:00-07:00,2022-08-03 08:05:00-07:00,2022-08-03 07:02:00-07:00,2022-08-03 09:47:00-07:00,0,0,0,95.0,165.0,120.0,380.0,0.002472,102.0,0.252170
1,DL,ATL,"Atlanta, GA",MSP,"Minneapolis, MN",8/3/2022,2262,1007,1140,DL,MSP,"Minneapolis, MN",SEA,"Seattle, WA",8/3/2022,890,1616,1800,8/3/2022,1816,2005,US/Eastern,US/Central,US/Central,US/Pacific,2022-08-03 06:07:00-04:00,2022-08-03 06:40:00-05:00,2022-08-03 11:16:00-05:00,2022-08-03 11:00:00-07:00,2022-08-03 13:16:00-05:00,2022-08-03 13:05:00-07:00,0,0,0,93.0,104.0,276.0,473.0,0.098992,125.0,12.373948
2,DL,ATL,"Atlanta, GA",SLC,"Salt Lake City, UT",8/3/2022,381,810,1012,DL,SLC,"Salt Lake City, UT",SEA,"Seattle, WA",8/3/2022,500,1140,1254,8/3/2022,1530,1640,US/Eastern,US/Mountain,US/Mountain,US/Pacific,2022-08-03 04:10:00-04:00,2022-08-03 04:12:00-06:00,2022-08-03 05:40:00-06:00,2022-08-03 05:54:00-07:00,2022-08-03 09:30:00-06:00,2022-08-03 09:40:00-07:00,0,0,0,122.0,74.0,88.0,284.0,0.067143,226.0,15.174248
3,DL,ATL,"Atlanta, GA",MSP,"Minneapolis, MN",8/3/2022,466,855,1030,DL,MSP,"Minneapolis, MN",SEA,"Seattle, WA",8/3/2022,890,1616,1800,8/3/2022,1816,2005,US/Eastern,US/Central,US/Central,US/Pacific,2022-08-03 04:55:00-04:00,2022-08-03 05:30:00-05:00,2022-08-03 11:16:00-05:00,2022-08-03 11:00:00-07:00,2022-08-03 13:16:00-05:00,2022-08-03 13:05:00-07:00,0,0,0,95.0,104.0,346.0,545.0,0.209568,125.0,26.196057
4,DL,ATL,"Atlanta, GA",SJC,"San Jose, CA",8/3/2022,1743,910,1102,DL,SJC,"San Jose, CA",SEA,"Seattle, WA",8/3/2022,3608,1155,1414,8/3/2022,1431,1645,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2022-08-03 05:10:00-04:00,2022-08-03 04:02:00-07:00,2022-08-03 04:55:00-07:00,2022-08-03 07:14:00-07:00,2022-08-03 07:31:00-07:00,2022-08-03 09:45:00-07:00,0,0,0,112.0,139.0,53.0,304.0,0.203615,151.0,30.745851
5,DL,ATL,"Atlanta, GA",LAS,"Las Vegas, NV",8/3/2022,941,958,1104,DL,LAS,"Las Vegas, NV",SEA,"Seattle, WA",8/3/2022,2760,1330,1610,8/3/2022,1541,1818,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2022-08-03 05:58:00-04:00,2022-08-03 04:04:00-07:00,2022-08-03 06:30:00-07:00,2022-08-03 09:10:00-07:00,2022-08-03 08:41:00-07:00,2022-08-03 11:18:00-07:00,0,0,0,66.0,160.0,146.0,372.0,0.253587,128.0,32.459199
6,DL,ATL,"Atlanta, GA",SFO,"San Francisco, CA",8/3/2022,824,1000,1203,DL,SFO,"San Francisco, CA",SEA,"Seattle, WA",8/3/2022,3609,1304,1515,8/3/2022,1644,1843,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2022-08-03 06:00:00-04:00,2022-08-03 05:03:00-07:00,2022-08-03 06:04:00-07:00,2022-08-03 08:15:00-07:00,2022-08-03 09:44:00-07:00,2022-08-03 11:43:00-07:00,0,0,0,123.0,131.0,61.0,315.0,0.157776,208.0,32.817413
7,DL,ATL,"Atlanta, GA",SAN,"San Diego, CA",8/3/2022,868,950,1112,DL,SAN,"San Diego, CA",SEA,"Seattle, WA",8/3/2022,2904,1355,1650,8/3/2022,1520,1811,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2022-08-03 05:50:00-04:00,2022-08-03 04:12:00-07:00,2022-08-03 06:55:00-07:00,2022-08-03 09:50:00-07:00,2022-08-03 08:20:0

You can also order by trip duration as shown:

In [4]:
df = itineraryBuilder('faa', origin, destination, flight_date, tc, dne, dnl, ane, anl, orderby='duration'); df.head(10)

,FIRST_LEG_AIRLINE,FIRST_LEG_ORIG,FIRST_LEG_ORIG_CITY,FIRST_LEG_DEST,FIRST_LEG_DEST_CITY,FIRST_LEG_DATE,FIRST_LEG_FLIGHT_NUM,FIRST_LEG_DEP_TIME,FIRST_LEG_ARR_TIME,SECOND_LEG_AIRLINE,SECOND_LEG_ORIG,SECOND_LEG_ORIG_CITY,SECOND_LEG_DEST,SECOND_LEG_DEST_CITY,SECOND_LEG_DATE,SECOND_LEG_FLIGHT_NUM,SECOND_LEG_DEP_TIME,SECOND_LEG_ARR_TIME,NEXT_BEST_SECOND_LEG_DATE,NEXT_BEST_SECOND_LEG_DEP_TIME,NEXT_BEST_SECOND_LEG_ARR_TIME,FIRST_LEG_ORIG_TZ,FIRST_LEG_DEST_TZ,SECOND_LEG_ORIG_TZ,SECOND_LEG_DEST_TZ,FIRST_LEG_DEP_TIMESTAMP,FIRST_LEG_ARR_TIMESTAMP,SECOND_LEG_DEP_TIMESTAMP,SECOND_LEG_ARR_TIMESTAMP,NEXT_BEST_SECOND_LEG_DEP_TIMESTAMP,NEXT_BEST_SECOND_LEG_ARR_TIMESTAMP,overnight_bool_1,overnight_bool_2,overnight_bool_3,FIRST_FLIGHT_DURATION,SECOND_FLIGHT_DURATION,CONNECT_TIME,TRIP_TIME,RISK_MISSED_CONNECTION,NEXT_FLIGHT_TIMELOSS,TOTAL_RISK
0,DL,ATL,"Atlanta, GA",DEN,"Denver, CO",8/3/2022,1537,955,1059,DL,DEN,"Denver, CO",SEA,"Seattle, WA",8/3/2022,3551,1230,1436,8/4/2022,650,848,US/Eastern,US/Mountain,US/Mountain,US/Pacific,2022-08-03 05:55:00-04:00,2022-08-03 04:59:00-06:00,2022-08-03 06:30:00-06:00,2022-08-03 07:36:00-07:00,2022-08-04 00:50:00-06:00,2022-08-04 01:48:00-07:00,0,0,0,64.0,126.0,91.0,281.0,0.053024,1092.0,57.902071
1,DL,ATL,"Atlanta, GA",SLC,"Salt Lake City, UT",8/3/2022,381,810,1012,DL,SLC,"Salt Lake City, UT",SEA,"Seattle, WA",8/3/2022,500,1140,1254,8/3/2022,1530,1640,US/Eastern,US/Mountain,US/Mountain,US/Pacific,2022-08-03 04:10:00-04:00,2022-08-03 04:12:00-06:00,2022-08-03 05:40:00-06:00,2022-08-03 05:54:00-07:00,2022-08-03 09:30:00-06:00,2022-08-03 09:40:00-07:00,0,0,0,122.0,74.0,88.0,284.0,0.319608,226.0,72.231373
2,DL,ATL,"Atlanta, GA",ONT,"Ontario, CA",8/3/2022,1144,953,1110,DL,ONT,"Ontario, CA",SEA,"Seattle, WA",8/3/2022,3659,1206,1445,8/4/2022,613,855,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2022-08-03 05:53:00-04:00,2022-08-03 04:10:00-07:00,2022-08-03 05:06:00-07:00,2022-08-03 07:45:00-07:00,2022-08-03 23:13:00-07:00,2022-08-04 01:55:00-07:00,0,0,0,77.0,159.0,56.0,292.0,0.042663,1090.0,46.502641
3,DL,ATL,"Atlanta, GA",GEG,"Spokane, WA",8/3/2022,921,950,1127,DL,GEG,"Spokane, WA",SEA,"Seattle, WA",8/3/2022,3605,1325,1443,8/3/2022,1725,1839,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2022-08-03 05:50:00-04:00,2022-08-03 04:27:00-07:00,2022-08-03 06:25:00-07:00,2022-08-03 07:43:00-07:00,2022-08-03 10:25:00-07:00,2022-08-03 11:39:00-07:00,0,0,0,97.0,78.0,118.0,293.0,0.538606,236.0,127.111022
4,DL,ATL,"Atlanta, GA",DTW,"Detroit, MI",8/3/2022,352,910,1107,DL,DTW,"Detroit, MI",SEA,"Seattle, WA",8/3/2022,830,1215,1410,8/3/2022,1630,1821,US/Eastern,US/Eastern,US/Eastern,US/Pacific,2022-08-03 05:10:00-04:00,2022-08-03 07:07:00-04:00,2022-08-03 08:15:00-04:00,2022-08-03 07:10:00-07:00,2022-08-03 12:30:00-04:00,2022-08-03 11:21:00-07:00,0,0,0,117.0,115.0,68.0,300.0,0.825738,251.0,207.260300
5,DL,ATL,"Atlanta, GA",SJC,"San Jose, CA",8/3/2022,1743,910,1102,DL,SJC,"San Jose, CA",SEA,"Seattle, WA",8/3/2022,3608,1155,1414,8/3/2022,1431,1645,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2022-08-03 05:10:00-04:00,2022-08-03 04:02:00-07:00,2022-08-03 04:55:00-07:00,2022-08-03 07:14:00-07:00,2022-08-03 07:31:00-07:00,2022-08-03 09:45:00-07:00,0,0,0,112.0,139.0,53.0,304.0,0.260067,151.0,39.270125
6,DL,ATL,"Atlanta, GA",SNA,"Santa Ana, CA",8/3/2022,778,954,1123,DL,SNA,"Santa Ana, CA",SEA,"Seattle, WA",8/3/2022,1525,1226,1506,8/4/2022,730,1027,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2022-08-03 05:54:00-04:00,2022-08-03 04:23:00-07:00,2022-08-03 05:26:00-07:00,2022-08-03 08:06:00-07:00,2022-08-04 00:30:00-07:00,2022-08-04 03:27:00-07:00,0,0,0,89.0,160.0,63.0,312.0,0.362877,1161.0,421.300357
7,DL,ATL,"Atlanta, GA",SFO,"San Francisco, CA",8/3/2022,824,1000,1203,DL,SFO,"San Francisco, CA",SEA,"Seattle, WA",8/3/2022,3609,1304,1515,8/3/2022,1644,1843,US/Eastern,US/Pacific,US/Pacific,US/Pacific,2022-08-03 06:00:00-04:00,2022-08-03 05:03:00-07:00,2022-08-03 06:04:00-07:00,2022-08-03 08:15:00-07:00,2022-08-03 09:44:00-07:00,2022-08-03 11:43:00-

### Query Flights Function Call

Query flights includes many of the same parameters as Itinerary Builder and is in fact called by Itinerary Builder. This returns the initial query with the dates and times as string values.

In [5]:
origin = 'ATL'
destination = 'SEA'
flight_date = '8/3/2022'

dne = '700'
dnl = '1000'
ane = '1'
anl = '2000'

df = queryFlights('faa', origin, destination, flight_date, dne, dnl, ane, anl); df

,FIRST_LEG_AIRLINE,FIRST_LEG_ORIG,FIRST_LEG_ORIG_CITY,FIRST_LEG_DEST,FIRST_LEG_DEST_CITY,FIRST_LEG_DATE,FIRST_LEG_FLIGHT_NUM,FIRST_LEG_DEP_TIME,FIRST_LEG_ARR_TIME,SECOND_LEG_AIRLINE,SECOND_LEG_ORIG,SECOND_LEG_ORIG_CITY,SECOND_LEG_DEST,SECOND_LEG_DEST_CITY,SECOND_LEG_DATE,SECOND_LEG_FLIGHT_NUM,SECOND_LEG_DEP_TIME,SECOND_LEG_ARR_TIME,NEXT_BEST_SECOND_LEG_DATE,NEXT_BEST_SECOND_LEG_DEP_TIME,NEXT_BEST_SECOND_LEG_ARR_TIME
0,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",8/3/2022,2725,820,937,DL,AUS,"Austin, TX",SEA,"Seattle, WA",8/3/2022,988,600,825,8/3/2022,1628,1852
1,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",8/3/2022,2725,820,937,DL,AUS,"Austin, TX",SEA,"Seattle, WA",8/3/2022,1067,1628,1852,8/4/2022,600,825
2,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",8/3/2022,2725,820,937,DL,AUS,"Austin, TX",SEA,"Seattle, WA",8/4/2022,988,600,825,8/4/2022,1628,1852
3,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",8/3/2022,2725,820,937,DL,AUS,"Austin, TX",SEA,"Seattle, WA",8/4/2022,1067,1628,1852,8/5/2022,600,825
4,DL,ATL,"Atlanta, GA",AUS,"Austin, TX",8/3/2022,2725,820,937,DL,AUS,"Austin, TX",SEA,"Seattle, WA",8/5/2022,988,600,825,8/5/2022,1628,1852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,DL,ATL,"Atlanta, GA",SNA,"Santa Ana, CA",8/3/2022,778,954,1123,DL,SNA,"Santa Ana, CA",SEA,"Seattle, WA",8/5/2022,2557,730,1027,8/5/2022,1226,1506
304,DL,ATL,"Atlanta, GA",TPA,"Tampa, FL",8/3/2022,1532,1000,1128,DL,TPA,"Tampa, FL",SEA,"Seattle, WA",8/3/2022,459,1106,1401,8/4/2022,1106,1401
305,DL,ATL,"Atlanta, GA",TPA,"Tampa, FL",8/3/2022,2211,840,1007,DL,TPA,"Tampa, FL",SEA,"Seattle, WA",8/3/2022,459,1106,1401,8/4/2022,1106,1401
306,DL,ATL,"Atlanta, GA",TPA,"Tampa, FL",8/3/2022,1532,1000,1128,DL,TPA,"Tampa, FL",SEA,"Seattle, WA",8/4/2022,459,1106,1401,8/5/2022,1106,1401


### getValidDestinations function call

Use this function to get a list of locations which can be reached with exactly two flights from the origin city, within a two day period.

In [6]:
df2 = getValidDestinations('faa', origin, flight_date); df2

,AIRPORT,CITY
0,ABR,"Aberdeen, SD"
1,ALB,"Albany, NY"
2,ABQ,"Albuquerque, NM"
3,ABE,"Allentown/Bethlehem/Easton, PA"
4,APN,"Alpena, MI"
...,...,...
169,HPN,"White Plains, NY"
170,ICT,"Wichita, KS"
171,XWA,"Williston, ND"
172,ILM,"Wilmington, NC"
